In [1]:
!pip install q numpy==1.17

In [0]:
import re

import os
from os import listdir
from os.path import isfile, join

import torch
from torch.utils.data import Dataset

import torchvision
from torchvision import transforms
from torchvision import models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import numpy as np

from PIL import Image

import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt

In [3]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1p1BWofDJOKXqCtO0JPT5VyuIPOsuxOuj' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1p1BWofDJOKXqCtO0JPT5VyuIPOsuxOuj" -O openlogo.tar && rm -rf /tmp/cookies.txt
!tar -xvf openlogo.tar

images_folder = "openlogo/JPEGImages"
annotations_folder = "openlogo/Annotations"
train_query_file = "openlogo/ImageSets/Main/train_test/train_all.txt"
test_query_file = "openlogo/ImageSets/Main/train_test/test_all.txt"

Streaming output truncated to the last 5000 lines.
openlogo/JPEGImages/hondaimg000205.jpg
openlogo/Annotations/apeimg000015.xml
openlogo/Annotations/siemens3.xml
openlogo/JPEGImages/logos32plus_003856.jpg
openlogo/JPEGImages/chanel41.jpg
openlogo/JPEGImages/Belgalogos_07582934.jpg
openlogo/JPEGImages/61839900.jpg
openlogo/Annotations/kraftimg000003.xml
openlogo/JPEGImages/logos32plus_000801.jpg
openlogo/Annotations/logos32plus_003692.xml
openlogo/JPEGImages/essoimg000006.jpg
openlogo/JPEGImages/logos32plus_001952.jpg
openlogo/Annotations/batman5.xml
openlogo/JPEGImages/Hisense_sportslogo_65.jpg
openlogo/Annotations/logos32plus_000073.xml
openlogo/JPEGImages/flickrlogo27_3778060828.jpg
openlogo/Annotations/boschimg000151.xml
openlogo/JPEGImages/aralimg000145.jpg
openlogo/Annotations/prada48.xml
openlogo/JPEGImages/fritolaysimg000043.jpg
openlogo/Annotations/logos32plus_000459.xml
openlogo/Annotations/logos32plus_004264.xml
openlogo/JPEGImages/boeingimg000044.jpg
openlogo/JPEGImages/logo

In [4]:
%%shell

git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 8318 (delta 5), reused 5 (delta 0), pack-reused 8297
Receiving objects: 100% (8318/8318), 10.24 MiB | 6.44 MiB/s, done.
Resolving deltas: 100% (5706/5706), done.
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be37608 version check against PyTorch's CUDA version


In [0]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
image_pattern = re.compile(r'^(?P<key>\w+)\.jpg$')
annotation_pattern = re.compile(r'^(?P<key>\w+)\.xml$')

class LogoDetectionDataset(Dataset):
  
  def __init__(self, image_folder, annotation_folder, query_file, transform=None, target_transform=None):
    self.image_folder = image_folder
    self.annotation_folder = annotation_folder
    self.transform = transform
    self.target_transform = target_transform
    self.image_names = self.__get_file_names__(image_folder, image_pattern)
    self.annotation_names = self.__get_file_names__(annotation_folder, annotation_pattern)

    self.keys = sorted(list(set(self.image_names.keys()) & set(self.annotation_names.keys())))
    with open(query_file) as file:
      file_content = file.read()
      self.keys = list(filter(lambda key: key in file_content, self.keys))
  
  def __len__(self):
    return len(self.keys)

  def __getitem__(self, index): 
    key = self.keys[index]

    image_name = self.image_names[key]
    annotation_name = self.annotation_names[key]

    image_path = os.path.join(self.image_folder, image_name)
    annotation_path = os.path.join(self.annotation_folder, annotation_name)

    img = Image.open(image_path).convert('RGB')
    target = self.__get_target__(annotation_path, index)

    if self.transform:
      img = self.transform(img)

    if self.target_transform:
      img, target = self.target_transform(img, target)

    #img = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(img)  

    return img, target

  def __get_target__(self, annotation_path, index):
    target = {}
    boxes = []
    root = ET.parse(annotation_path).getroot()
    for bndbox in root.findall('object/bndbox'):
      xmin = int(bndbox[0].text)
      ymin = int(bndbox[1].text)
      xmax = int(bndbox[2].text)
      ymax = int(bndbox[3].text)
      boxes.append((xmin, ymin, xmax, ymax))

    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    num_objs = len(boxes)
    labels = torch.ones((num_objs), dtype=torch.int64)
    img_id = torch.tensor([index])
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

    target['boxes'] = boxes
    target['labels'] = labels
    target['image_id'] = img_id
    target['area'] = area
    target['iscrowd'] = iscrowd
    return target

  def __get_file_names__(self, folder, file_name_pattern):
    file_names = {}
    for f in listdir(folder):
      match = file_name_pattern.match(f)
      if not match:
        continue
      
      num = match.group('key')
      if num == '':
        continue
      
      file_names[num] = f

    return file_names

In [0]:
from engine import train_one_epoch, evaluate
import utils
import transforms as detectionTransforms

trn = transforms.Compose([
                          transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
                          transforms.RandomGrayscale(p=0.1)
])

target_trn = detectionTransforms.Compose([
                                detectionTransforms.ToTensor(),
                                detectionTransforms.RandomHorizontalFlip(0.5)
])

test_target_trn = detectionTransforms.Compose([
                                detectionTransforms.ToTensor(),
])

num_classes = 2
batch_size = 4

dataset = LogoDetectionDataset(images_folder, annotations_folder, train_query_file, transform=trn, target_transform=target_trn)
test_dataset = LogoDetectionDataset(images_folder, annotations_folder, test_query_file, target_transform=test_target_trn)
orig_test_dataset = LogoDetectionDataset(images_folder, annotations_folder, test_query_file)

validation_size = .0

data_size = len(dataset)
test_data_size = len(test_dataset)

split_val = int(np.floor(validation_size * data_size))

indices = list(range(data_size))
test_indices = list(range(test_data_size))
np.random.seed(42)
np.random.shuffle(indices)
np.random.shuffle(test_indices)
val_indices, train_indices = indices[:split_val], indices[split_val:]

train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_indices)
test_sampler = torch.utils.data.sampler.SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler, collate_fn=utils.collate_fn)
val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                         sampler=val_sampler, collate_fn=utils.collate_fn)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1,
                                          sampler=test_sampler, collate_fn=utils.collate_fn)

In [0]:
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

params = (p for p in model.parameters() if p.requires_grad)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.5, step_size=2)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [8]:
model = torch.load("drive/My Drive/save/model")
model.to(device)

params = (p for p in model.parameters() if p.requires_grad)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.5, step_size=1)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torchvision.models.detection.faster_rcnn.FasterRCNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torchvision.models.detection.transform.GeneralizedRCNNTransform' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by ac

In [0]:
evaluate(model, test_loader, device=device)

creating index...
index created!


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Test:  [   0/8322]  eta: 1:17:31  model_time: 0.5156 (0.5156)  evaluator_time: 0.0116 (0.0116)  time: 0.5589  data: 0.0308  max mem: 605
Test:  [ 100/8322]  eta: 0:13:39  model_time: 0.0671 (0.0732)  evaluator_time: 0.0025 (0.0031)  time: 0.0899  data: 0.0167  max mem: 741
Test:  [ 200/8322]  eta: 0:13:29  model_time: 0.0743 (0.0713)  evaluator_time: 0.0025 (0.0031)  time: 0.0927  data: 0.0167  max mem: 749
Test:  [ 300/8322]  eta: 0:13:29  model_time: 0.0696 (0.0709)  evaluator_time: 0.0025 (0.0031)  time: 0.0870  data: 0.0138  max mem: 812
Test:  [ 400/8322]  eta: 0:13:08  model_time: 0.0683 (0.0706)  evaluator_time: 0.0029 (0.0034)  time: 0.1077  data: 0.0298  max mem: 812
Test:  [ 500/8322]  eta: 0:12:48  model_time: 0.0742 (0.0705)  evaluator_time: 0.0029 (0.0034)  time: 0.1061  data: 0.0275  max mem: 812
Test:  [ 600/8322]  eta: 0:12:29  model_time: 0.0736 (0.0703)  evaluator_time: 0.0029 (0.0033)  time: 0.0929  data: 0.0163  max mem: 812
Test:  [ 700/8322]  eta: 0:12:14  model_t

In [0]:
checkpoint = torch.load("drive/My Drive/save/save-aug-0-checkponit.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

In [0]:
last_saved_epoch = 0
num_epochs = 3

for epoch in range(num_epochs):
  train_one_epoch(model, optimizer, train_loader, device, last_saved_epoch + epoch, print_freq=1000)
  lr_scheduler.step()
  torch.save({'model_state_dict': model.state_dict(),                                                 
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': lr_scheduler.state_dict()}, "drive/My Drive/save/save-aug-model" + str(last_saved_epoch + epoch) + "-checkponit.pth") 
  torch.save(model, "drive/My Drive/save/save-aug-model" + str(last_saved_epoch + epoch))

torch.save(model, "drive/My Drive/save/save-aug-model-complete")

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch: [0]  [   0/4634]  eta: 7:45:07  lr: 0.000001  loss: 0.1918 (0.1918)  loss_classifier: 0.1121 (0.1121)  loss_box_reg: 0.0587 (0.0587)  loss_objectness: 0.0081 (0.0081)  loss_rpn_box_reg: 0.0129 (0.0129)  time: 6.0224  data: 0.6130  max mem: 7307
Epoch: [0]  [1000/4634]  eta: 4:31:34  lr: 0.000500  loss: 0.0861 (0.1167)  loss_classifier: 0.0455 (0.0615)  loss_box_reg: 0.0218 (0.0289)  loss_objectness: 0.0072 (0.0116)  loss_rpn_box_reg: 0.0083 (0.0147)  time: 4.3567  data: 0.4905  max mem: 8405


In [0]:
model = torch.load("drive/My Drive/save/save-aug-complete")
evaluate(model, test_loader, device=device)

creating index...
index created!


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Test:  [   0/8322]  eta: 2:01:32  model_time: 0.8074 (0.8074)  evaluator_time: 0.0153 (0.0153)  time: 0.8763  data: 0.0450  max mem: 575
Test:  [ 100/8322]  eta: 0:50:37  model_time: 0.3576 (0.3451)  evaluator_time: 0.0036 (0.0049)  time: 0.3746  data: 0.0159  max mem: 655
Test:  [ 200/8322]  eta: 0:49:37  model_time: 0.3451 (0.3400)  evaluator_time: 0.0038 (0.0049)  time: 0.3810  data: 0.0372  max mem: 722
Test:  [ 300/8322]  eta: 0:49:15  model_time: 0.3317 (0.3411)  evaluator_time: 0.0038 (0.0049)  time: 0.3617  data: 0.0189  max mem: 732
Test:  [ 400/8322]  eta: 0:48:20  model_time: 0.3300 (0.3390)  evaluator_time: 0.0038 (0.0048)  time: 0.3492  data: 0.0163  max mem: 732
Test:  [ 500/8322]  eta: 0:47:41  model_time: 0.3520 (0.3385)  evaluator_time: 0.0038 (0.0048)  time: 0.3633  data: 0.0170  max mem: 732
Test:  [ 600/8322]  eta: 0:47:05  model_time: 0.3363 (0.3387)  evaluator_time: 0.0039 (0.0048)  time: 0.3666  data: 0.0193  max mem: 732
Test:  [ 700/8322]  eta: 0:46:27  model_t

In [0]:
def get_iou(bb1, bb2):
  bb1_x1 = int(bb1[0])
  bb1_y1 = int(bb1[1])
  bb1_x2 = int(bb1[2])
  bb1_y2 = int(bb1[3])
  bb2_x1 = int(bb2[0])
  bb2_y1 = int(bb2[1])
  bb2_x2 = int(bb2[2])
  bb2_y2 = int(bb2[3])
  assert bb1_x1 < bb1_x2
  assert bb1_y1 < bb1_y2
  assert bb2_x1 < bb2_x2
  assert bb2_y1 < bb2_y2

  x_left = max(bb1_x1, bb2_x1)
  y_top = max(bb1_y1, bb2_y1)
  x_right = min(bb1_x2, bb2_x2)
  y_bottom = min(bb1_y2, bb2_y2)

  display(str(x_left) + " " + str(y_top) + " " + str(x_right) + " " + str(y_bottom))

  if x_right < x_left or y_bottom < y_top:
    return 0.0

  intersection_area = (x_right - x_left) * (y_bottom - y_top)

  bb1_area = (bb1_x2 - bb1_x1) * (bb1_y2 - bb1_y1)
  bb2_area = (bb2_x2 - bb2_x1) * (bb2_y2 - bb2_y1)

  iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
  assert iou >= 0.0
  assert iou <= 1.0
  return iou